# SL Vehicle Price Prediciton

# Load Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix,classification_report
from catboost import CatBoostClassifier, CatBoostRegressor
# from sklearn.model_selection import GridSearchCV

import seaborn as sns
from matplotlib import pyplot

## Load Data

In [2]:
# Read Data
# Change the line to local dir path
df = pd.read_csv("vehicle_data.csv")

C:\Users\Nidula\AppData\Local\Temp\ipykernel_18636\813459222.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("vehicle_data.csv")


In [3]:
df.head()

,Title,Sub_title,Price,Brand,Model,Edition,Year,Condition,Transmission,Body,Fuel,Capacity,Mileage,Location,Description,Post_URL,Seller_name,Seller_type,published_date
0,Honda Vezel Limited 2017 for sale,"Posted on 01 Sep 2:45 pm, Kurunegala City, Kur...","Rs 12,900,000",Honda,Vezel,Limited,2017,Used,Automatic,SUV / 4x4,Hybrid,"1,500 cc","37,000 km","Kurunegala City, Kurunegala",HONDA VEZEL LIMITEDCBE-XXXXMANU YEAR : 2017LIM...,https://ikman.lk/en/ad/honda-vezel-limited-201...,Prasad Enterprises,Premium-Member,2022-09-01 14:45:00
1,Jaguar XF Fully Loaded 2013 for sale,"Posted on 28 Sep 10:23 am, Kurunegala City, Ku...","Rs 16,250,000",Jaguar,XF,Fully Loaded,2013,Used,Automatic,Saloon,Diesel,"2,200 cc","42,000 km","Kurunegala City, Kurunegala",1st owner(registered owner)Diesel(13-14km per ...,https://ikman.lk/en/ad/jaguar-xf-fully-loaded-...,Dhanushka Dharmasri,Member,2022-09-28 10:23:00
2,Toyota CHR GT New Face 2wd 2020 for sale,"Posted on 16 Sep 1:16 pm, Colombo 7, Colombo","Rs 17,000,000",Toyota,CHR,GT New Face 2wd,2020,Used,Automatic,SUV / 4x4,Petrol,"1,200 cc","15,000 km","Colombo 7, Colombo",2020 Facelift modelFirst owner (No previous ow...,https://ikman.lk/en/ad/toyota-chr-gt-new-face-...,Carzone Lanka,Member,2022-09-16 13:16:00
3,Toyota CHR Toyato 2017 for sale,"Posted on 30 Sep 10:12 am, Colombo 7, Colombo","Rs 13,000,000",Toyota,CHR,Toyato,2017,Used,Automatic,SUV / 4x4,Petrol,"1,200 cc","21,000 km","Colombo 7, Colombo",Toyota CHR GT Turbo1st owner Imported as a bra...,https://ikman.lk/en/ad/toyota-chr-toyato-2017-...,Carzone Lanka,Member,2022-09-30 10:12:00
4,Toyota Land Cruiser Prado 2015 for sale,"Posted on 30 Sep 9:50 pm, Kelaniya, Gampaha","Rs 33,500,000",Toyota,Land Cruiser Prado,prado,2015,Used,Automatic,SUV / 4x4,Petrol,"2,700 cc","57,000 km","Kelaniya, Gampaha",Toyota Land Cruiser Prado 20152 nd Owner for 5...,https://ikman.lk/en/ad/toyota-land-cruiser-pra...,Shine Music Center,Member,2022-09-30 21:50:00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123971 entries, 0 to 123970
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Title           123971 non-null  object
 1   Sub_title       123971 non-null  object
 2   Price           123971 non-null  object
 3   Brand           123971 non-null  object
 4   Model           123970 non-null  object
 5   Edition         47538 non-null   object
 6   Year            123971 non-null  int64 
 7   Condition       123971 non-null  object
 8   Transmission    123970 non-null  object
 9   Body            118876 non-null  object
 10  Fuel            123971 non-null  object
 11  Capacity        123766 non-null  object
 12  Mileage         123971 non-null  object
 13  Location        123971 non-null  object
 14  Description     107797 non-null  object
 15  Post_URL        123971 non-null  object
 16  Seller_name     123873 non-null  object
 17  Seller_type     123971 non-nu

In [5]:
df.describe()

,Year
count,123971.000000
mean,2008.409765
std,25.330835
min,0.000000
25%,2004.000000
50%,2013.000000
75%,2016.000000
max,2561.000000


In [6]:
df.isnull().sum()

Title                 0
Sub_title             0
Price                 0
Brand                 0
Model                 1
Edition           76433
Year                  0
Condition             0
Transmission          1
Body               5095
Fuel                  0
Capacity            205
Mileage               0
Location              0
Description       16174
Post_URL              0
Seller_name          98
Seller_type           0
published_date        0
dtype: int64

## Data Preprocessing

In [7]:
temp = df.copy()

In [8]:
df = temp.copy()

In [9]:
# Remove Duplicates
df = df.drop_duplicates(subset='Post_URL', keep="first")

In [10]:
# Remove Unnecasary Fields
selected_columns = ['Price', 'Brand', 'Model', 'Year', 'Edition',
       'Condition', 'Transmission', 'Body', 'Fuel', 'Capacity', 'Mileage', 'Seller_type']

df = df[selected_columns]

In [11]:
# Fill Missing Values with appripriate values
df["Fuel"] = df["Fuel"].fillna("Petrol")
df["Seller_type"] = df["Seller_type"].fillna("Member")
df["Edition"] = df["Edition"].fillna("standard")

In [12]:
# Extract Numeric Columns
def extract_num(row):
    amount = ""
    for w in str(row):
        if w.isnumeric():
            amount += w
    if amount == "":
        return None
    return int(amount)
    
df["Price"] = df["Price"].apply(extract_num)
df["Mileage"] = df["Mileage"].apply(extract_num)
df["Capacity"] = df["Capacity"].apply(extract_num)

In [13]:
# No need to scale or encode the data because caboost automatically scales and applies label encoding

In [14]:
df = df.dropna()

In [15]:
X = df.drop("Price", axis=1)
y = df["Price"]


In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97927 entries, 0 to 123910
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         97927 non-null  object 
 1   Model         97927 non-null  object 
 2   Year          97927 non-null  int64  
 3   Edition       97927 non-null  object 
 4   Condition     97927 non-null  object 
 5   Transmission  97927 non-null  object 
 6   Body          97927 non-null  object 
 7   Fuel          97927 non-null  object 
 8   Capacity      97927 non-null  float64
 9   Mileage       97927 non-null  float64
 10  Seller_type   97927 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 9.0+ MB


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

## Build Catoost Model

In [18]:
param_grid = {'depth': 15, 'learning_rate': 0.04, 'od_type': 'Iter', 'od_type':"Iter",'od_wait': 10, 'task_type':"GPU"}
cat = CatBoostRegressor(**param_grid)

In [19]:
cat_features = ['Brand', 'Model', 'Condition', 'Transmission', 'Edition',
       'Body', 'Seller_type', 'Fuel']

X.columns

Index(['Brand', 'Model', 'Year', 'Edition', 'Condition', 'Transmission',
       'Body', 'Fuel', 'Capacity', 'Mileage', 'Seller_type'],
      dtype='object')

In [20]:
cat.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features=cat_features)

0:	learn: 10552399.8113207	test: 9726061.1143985	best: 9726061.1143985 (0)	total: 541ms	remaining: 9m
1:	learn: 10311532.0020717	test: 9454938.6138207	best: 9454938.6138207 (1)	total: 979ms	remaining: 8m 8s
2:	learn: 10090232.9319103	test: 9202354.0814544	best: 9202354.0814544 (2)	total: 1.41s	remaining: 7m 49s
3:	learn: 9877944.3688400	test: 8957525.0713468	best: 8957525.0713468 (3)	total: 1.83s	remaining: 7m 36s
4:	learn: 9677111.5116556	test: 8726860.7587053	best: 8726860.7587053 (4)	total: 2.26s	remaining: 7m 30s
5:	learn: 9483961.8102797	test: 8502809.3288576	best: 8502809.3288576 (5)	total: 2.72s	remaining: 7m 30s
6:	learn: 9301531.4377773	test: 8291059.3493468	best: 8291059.3493468 (6)	total: 3.16s	remaining: 7m 28s
7:	learn: 9138761.4897352	test: 8096459.8303860	best: 8096459.8303860 (7)	total: 3.59s	remaining: 7m 24s
8:	learn: 8977093.3695397	test: 7904717.8859877	best: 7904717.8859877 (8)	total: 4.01s	remaining: 7m 22s
9:	learn: 8823721.6991288	test: 7728954.6594174	best: 772

78:	learn: 5776468.1472503	test: 4506413.5644783	best: 4506413.5644783 (78)	total: 33.9s	remaining: 6m 35s
79:	learn: 5771505.7186211	test: 4503472.2985706	best: 4503472.2985706 (79)	total: 34.3s	remaining: 6m 35s
80:	learn: 5763137.1177362	test: 4499379.2519168	best: 4499379.2519168 (80)	total: 34.8s	remaining: 6m 34s
81:	learn: 5757251.0028463	test: 4495702.3719110	best: 4495702.3719110 (81)	total: 35.2s	remaining: 6m 33s
82:	learn: 5738432.5827342	test: 4493240.5972358	best: 4493240.5972358 (82)	total: 35.6s	remaining: 6m 33s
83:	learn: 5735527.9035401	test: 4490552.6510774	best: 4490552.6510774 (83)	total: 36.1s	remaining: 6m 33s
84:	learn: 5735250.8411125	test: 4490232.8805360	best: 4490232.8805360 (84)	total: 36.4s	remaining: 6m 32s
85:	learn: 5729593.2847228	test: 4485628.1601478	best: 4485628.1601478 (85)	total: 36.9s	remaining: 6m 31s
86:	learn: 5721737.1132858	test: 4481848.2111952	best: 4481848.2111952 (86)	total: 37.3s	remaining: 6m 31s
87:	learn: 5721454.4769845	test: 4481

In [21]:
preds = cat.predict(X_test)

In [22]:
df_preds = pd.DataFrame({"Actual Price": y_test, "Predicted Price": preds})

In [23]:
df_preds

,Actual Price,Predicted Price
55424,3685000.0,4.127700e+06
83606,800000.0,1.583045e+06
55039,7250000.0,6.633335e+06
46380,11790000.0,9.323907e+06
36315,11000000.0,8.517092e+06
...,...,...
117158,2450000.0,2.018597e+06
38238,6800000.0,6.443296e+06
115732,360000.0,5.427628e+05
98920,6500000.0,3.000051e+06


In [24]:
from sklearn.metrics import mean_squared_error

# Root mean squared error 
print(mean_squared_error(df_preds["Actual Price"], df_preds["Predicted Price"])**0.5)

4417689.302911875
